In [10]:
import rssi

In [11]:
interface = 'wlp58s0'

In [78]:
name = "rssi"

from subprocess import Popen, PIPE # Used to run native OS commads in python wrapped subproccess
import numpy # Used for matrix operations in localization algorithm
from sys import version_info # Used to check the Python-interpreter version at runtime

# RSSI_Scan
    # Use:
        # from rssi import RSSI_Scan
        # rssi_scan_instance = RSSI_Scan('network_interface_name) 
    # -------------------------------------------------------
    # Description:
        # Allows a user to query all available accesspoints available.
        # User has the option of define a specific set of access 
        # points to query.
    # -------------------------------------------------------
    # Input: interface name
        # [ie. network interface names: wlp1s0m, docker0, wlan0] 
class RSSI_Scan(object):
    # Allows us to declare a network interface externally.
    def __init__(self, interface):
        self.interface = interface

    # getRawNetworkScan
        # Description:
            # Runs the Ubuntu command 'iwlist' to scan for available networks.
            # Returns the raw console window output (unparsed).
        # ----------------------------------------------------------------
        # Input: (optional) 
            #   sudo: bool; defaults to false. False will not refresh the 
            #         network interface upon query. Sudo=true will require 
            #         the user will need to enter a sudo password at runtime.
        # ----------------------------------------------------------------
        # Returns: Raw terminal output
            # {
            #     'output':'''wlp1s0    Scan completed :
            #   Cell 01 - Address: A0:3D:6F:26:77:8E
            #             Channel:144
            #             Frequency:5.72 GHz
            #             Quality=43/70  Signal level=-67 dBm  
            #             Encryption key:on
            #             ESSID:"ucrwpa"
            #             Bit Rates:24 Mb/s; 36 Mb/s; 48 Mb/s; 54 Mb/s
            #             Mode:Master
            #   Cell 02 - Address: A0:3D:6F:26:77:82
            #             Channel:1
            #             Frequency:2.412 GHz (Channel 1)
            #             Quality=43/70  Signal level=-67 dBm  
            #             Encryption key:on
            #             ESSID:"eduroam"
            #             Bit Rates:18 Mb/s; 24 Mb/s; 36 Mb/s; 48 Mb/s; 54 Mb/s
            #             Mode:Master''',
            #     'error':''
            # }
    def getRawNetworkScan(self, sudo=False):
        # Scan command 'iwlist interface scan' needs to be fed as an array.
        if sudo:
            scan_command = ['sudo','iwlist',self.interface,'scan']
        else:
            scan_command = ['iwlist',self.interface,'scan']
        # Open a subprocess running the scan command.
        scan_process = Popen(scan_command, stdout=PIPE, stderr=PIPE)
        # Returns the 'success' and 'error' output.
        (raw_output, raw_error) = scan_process.communicate() 
        # Block all execution, until the scanning completes.
        scan_process.wait()
        # Returns all output in a dictionary for easy retrieval.
        return {'output':raw_output,'error':raw_error}

    # getSSID
        # Description:
            # Parses the 'SSID' for a given cell.
        # -----------------------------------------------
        # Input: (Raw string)
            # 01 - Address: A0:3D:6F:26:77:8E
            # Channel:144
            # Frequency:5.72 GHz
            # Quality=43/70  Signal level=-67 dBm  
            # Encryption key:on
            # ESSID:"ucrwpa"
            # Bit Rates:24 Mb/s; 36 Mb/s; 48 Mb/s; 54 Mb/s
            # Mode:Master
        # -----------------------------------------------
        # Returns:
            # 'ucrwpa'
    @staticmethod
    def getSSID(raw_cell):
        ssid = raw_cell.split('ESSID:"')[1]
        ssid = ssid.split('"')[0]
        return ssid

    # getQuality
        # Description:
            # Parses 'Quality level' for a given cell.
        # -----------------------------------------------
        # Input: (Raw string)
            # 01 - Address: A0:3D:6F:26:77:8E
            # Channel:144
            # Frequency:5.72 GHz
            # Quality=43/70  Signal level=-67 dBm  
            # Encryption key:on
            # ESSID:"ucrwpa"
            # Bit Rates:24 Mb/s; 36 Mb/s; 48 Mb/s; 54 Mb/s
            # Mode:Master
        # -----------------------------------------------
        # Returns:
            # '43/70'
    @staticmethod
    def getFrequency(raw_cell):
        quality = raw_cell.split('Frequency:')[1]
        quality = quality.split(' ')[0]
        return float(quality)

    @staticmethod
    def getQuality(raw_cell):
        quality = raw_cell.split('Quality=')[1]
        quality = quality.split(' ')[0]
        return quality

    # getSignalLevel
        # Description:
            # Parses 'Signal level' for a given cell.
            # Measurement is in 'dBm'.
        # -----------------------------------------------
        # Input: (Raw string)
            # 01 - Address: A0:3D:6F:26:77:8E
            # Channel:144
            # Frequency:5.72 GHz
            # Quality=43/70  Signal level=-67 dBm  
            # Encryption key:on
            # ESSID:"ucrwpa"
            # Bit Rates:24 Mb/s; 36 Mb/s; 48 Mb/s; 54 Mb/s
            # Mode:Master
        # -----------------------------------------------
        # Returns: (string)
            # '-67'    
    @staticmethod
    def getSignalLevel(raw_cell):
        signal = raw_cell.split('Signal level=')[1]
        signal = int(signal.split(' ')[0])
        return signal

    # getMacAddress
        # Description:
            # Method returns the MAC address of the AP
        # -----------------------------------------------
        #   Input: (Raw string)
            # 01 - Address: A0:3D:6F:26:77:8E
            # Channel:144
            # Frequency:5.72 GHz
            # Quality=43/70  Signal level=-67 dBm  
            # Encryption key:on
            # ESSID:"ucrwpa"
            # Bit Rates:24 Mb/s; 36 Mb/s; 48 Mb/s; 54 Mb/s
            # Mode:Master
        # -----------------------------------------------
        # Returns: (string)
            #   'A0:3D:6F:26:77:8E'
    @staticmethod
    def getMacAddress(raw_cell):
        mac = raw_cell.split('Address: ')[1]
        mac = mac.split(' ')[0]
        mac = mac.strip()
        return mac

    # parseCell
        # Description:
            # Takes a raw cell string and parses it into a dictionary.
        # -----------------------------------------------
        # Input: (Raw string)
            # '''01 - Address: A0:3D:6F:26:77:8E
            # Channel:144
            # Frequency:5.72 GHz
            # Quality=43/70  Signal level=-67 dBm  
            # Encryption key:on
            # ESSID:"ucrwpa"
            # Bit Rates:24 Mb/s; 36 Mb/s; 48 Mb/s; 54 Mb/s
            # Mode:Master'''
        # -----------------------------------------------
        # Returns:
            # {
            #     'ssid':'ucrwpa',
            #     'quality':'43/70',
            #     'signal':'-67'
            # }    
    def parseCell(self, raw_cell):
        cell = {
            'ssid': self.getSSID(raw_cell),
            'quality': self.getQuality(raw_cell),
            'signal': self.getSignalLevel(raw_cell),
            'mac': self.getMacAddress(raw_cell),
            'frequency': self.getFrequency(raw_cell)
        }
        return cell

    # formatCells
        # Description:
            # Every network listed is considered a 'cell.
            # This function parses each cell into a dictionary.
            # Returns list of dictionaries. Makes use of 'parseCell'.
            # If not networks were detected, returns False.
        # -----------------------------------------------
        # Input: (Raw terminal string)
            # '''01 - Address: A0:3D:6F:26:77:8E
            # Channel:144
            # Frequency:5.72 GHz
            # Quality=43/70  Signal level=-67 dBm  
            # Encryption key:on
            # ESSID:"ucrwpa"
            # Bit Rates:24 Mb/s; 36 Mb/s; 48 Mb/s; 54 Mb/s
            # Mode:Master
            # 02 - Address: A0:3D:6F:26:77:8E
            # Channel:144
            # Frequency:5.72 GHz
            # Quality=30/70  Signal level=-42 dBm  
            # Encryption key:on
            # ESSID:"dd-wrt"
            # Bit Rates:24 Mb/s; 36 Mb/s; 48 Mb/s; 54 Mb/s
            # Mode:Master'''
        # -----------------------------------------------
        # Returns: (Array of dictionaries)
            # [
            #     {
            #         'ssid':'ucrwpa',
            #         'quality':'43/70',
            #         'signal':'-67'
            #     },
            #     {
            #         'ssid':'dd-wrt',
            #         'quality':'30/70',
            #         'signal':'-42'
            #     }
            # ]    
    def formatCells(self, raw_cell_string):
        try:
            raw_cell_string = raw_cell_string.decode()
        except:
            print('Already in string.')
        
        raw_cells = raw_cell_string.split('Cell') # Divide raw string into raw cells.
        raw_cells.pop(0) # Remove unneccesary "Scan Completed" message.
        if(len(raw_cells) > 0): # Continue execution, if atleast one network is detected.
            # Iterate through raw cells for parsing.
            # Array will hold all parsed cells as dictionaries.
            formatted_cells = [self.parseCell(cell) for cell in raw_cells]
            # Return array of dictionaries, containing cells.
            return formatted_cells
        else:
            print("Networks not detected.")
            return False
        # TODO implement function in ndoe to process this boolean (False)

    # filterAccessPoints
        # Description:
            # If the 'networks' parameter is passed to the 'getAPinfo'
            # function, then this method will filter out all irrelevant 
            # access-points. Access points specified in 'networks' array 
            # will be returned (if available).
        # -----------------------------------------------
        # Input: (Parsed array of cell dictionaries)
            # all_access_points = 
            # [
            #     {
            #         'ssid':'ucrwpa',
            #         'quality':'43/70',
            #         'signal':'-67'
            #     },
            #     {
            #         'ssid':'dd-wrt',
            #         'quality':'30/70',
            #         'signal':'-42'
            #     },
            #     {
            #         'ssid':'linksys',
            #         'quality':'58/70',
            #         'signal':'-24'
            #     }
            # ] 
            # network_names = (array of network names)
            # ['ucrwpa','dd-wrt']
        # -----------------------------------------------
        # Returns: (Array of dictionaries)
            # [
            #     {
            #         'ssid':'ucrwpa',
            #         'quality':'43/70',
            #         'signal':'-67'
            #     },
            #     {
            #         'ssid':'dd-wrt',
            #         'quality':'30/70',
            #         'signal':'-42'
            #     }
            # ] 
    @staticmethod
    def filterAccessPoints(all_access_points, network_names):
        focus_points = [] # Array holding the access-points of concern.
        # Iterate throguh all access-points found.
        for point in all_access_points:
            # Check if current AP is in our desired list.
            if point['ssid'] in network_names:
                focus_points.append(point)
        return focus_points
        # TODO implement something incase our desired ones were not found
 
    # getAPinfo
        # Description:
            # Method returns all (or chosen) available access points (in range).
            # Takes 2 optional parameters: 
            #   'networks' (array): 
            #       Lists all ssid's of concern. Will return only the available access 
            #       points listed here. If not provided, will return ALL access-points in range.        
            #   'sudo' (bool): 
            #       Whether of not method should use sudo privileges. If user uses sudo
            #       privileges, the network manager will be refreshed and will return 
            #       a fresh list of access-points available. If sudo is not provided, 
            #       a cached list will be returned. Cached list gets updated periodically.
        # -----------------------------------------------
        # Input: (Parsed array of cell dictionaries)
            # networks = (array of network names)
            # ['ucrwpa','dd-wrt']
            # sudo = True || False
        # -----------------------------------------------
        # Returns: (Array of dictionaries)
            # [
            #     {
            #         'ssid':'ucrwpa',
            #         'quality':'43/70',
            #         'signal':'-67'
            #     },
            #     {
            #         'ssid':'dd-wrt',
            #         'quality':'30/70',
            #         'signal':'-42'
            #     }
            # ] 
    def getAPinfo(self, networks=False, sudo=False):
        # TODO implement error callback if error is raise in subprocess
        # Unparsed access-point listing. AccessPoints are strings.
        raw_scan_output = self.getRawNetworkScan(sudo)['output']
        
        # Parsed access-point listing. Access-points are dictionaries.
        all_access_points = self.formatCells(raw_scan_output)
        # Checks if access-points were found.
        if all_access_points:
            # Checks if specific networks were declared.
            if networks:
                # Return specific access-points found.
                return self.filterAccessPoints(all_access_points, networks)
            else:
                # Return ALL access-points found.
                return all_access_points
        else:
            # No access-points were found. 
            return False


In [79]:
rssi_scanner = RSSI_Scan(interface)

In [80]:
rssi_scanner.getAPinfo()

[{'ssid': 'Sahu_5GHz',
  'quality': '49/70',
  'signal': -61,
  'mac': '78:17:35:C4:5E:DD',
  'frequency': 5.24}]

In [49]:
a = rssi_scanner.getRawNetworkScan()['output']

In [1]:
from src import point, vector, line, source
import numpy as np

In [2]:
sx, sy, sz = np.random.randint(0, 100, 1)[0], np.random.randint(0, 100, 1)[0],  np.random.randint(0, 100, 1)[0]

In [3]:
wifi = source.Source(sx, sy, sz)

In [4]:
initx, inity, initz = 0, 0, 0

In [5]:
start = point.Point(initx, inity, initz)

In [6]:
start.Intensity(wifi)

Intensity at x: 0, y: 0, z: 0, is I: 0.03005710850616171


0.03005710850616171

In [7]:
init_p1, init_p2 = point.Point(initx, inity, initz), point.Point(1, 1, initz)

In [8]:
init_line = line.Line(init_p1, init_p2, axis=2)

In [9]:
init_line.slope_xy, init_line.intercept_xy

(1.0, 0.0)

In [10]:
interation = 100
found = False
counter = 0
step = 1

inten1 = start.Intensity(wifi)
while found is False and counter < interation:
    
    
    start.MoveAlongLine_xy(init_line, step)
    inten2 = start.Intensity(wifi)
    
    if inten2 < inten1:
        step = -step/3
    
    if abs((inten2 - inten1) *100 / inten1) < 0.005:
        found = True
        
    inten1, inten2 = inten2, None
    
        
    

Intensity at x: 0, y: 0, z: 0, is I: 0.03005710850616171
Moved to x: 0.7071067811865476, y: 0.7071067811865475, z: 0
Intensity at x: 0.7071067811865476, y: 0.7071067811865475, z: 0, is I: 0.031023641849232845
Moved to x: 1.4142135623730951, y: 1.414213562373095, z: 0
Intensity at x: 1.4142135623730951, y: 1.414213562373095, z: 0, is I: 0.03203386473830186
Moved to x: 2.121320343559643, y: 2.1213203435596424, z: 0
Intensity at x: 2.121320343559643, y: 2.1213203435596424, z: 0, is I: 0.03309018019089575
Moved to x: 2.8284271247461903, y: 2.82842712474619, z: 0
Intensity at x: 2.8284271247461903, y: 2.82842712474619, z: 0, is I: 0.034195132915983424
Moved to x: 3.5355339059327378, y: 3.5355339059327373, z: 0
Intensity at x: 3.5355339059327378, y: 3.5355339059327373, z: 0, is I: 0.03535141627992259
Moved to x: 4.242640687119286, y: 4.242640687119285, z: 0
Intensity at x: 4.242640687119286, y: 4.242640687119285, z: 0, is I: 0.036561879149375
Moved to x: 4.949747468305834, y: 4.9497474683058

In [11]:
new_direction = init_line.GetPerpendicularFromPoint(start, axis=2)

In [12]:
new_direction.slope_xy, new_direction.intercept_xy

(-1.0, 74.01050976419194)

In [13]:
start.Goto(initx, inity, initz)

Moved to x: 0, y: 0, z: 0


In [14]:
interation = 100
found = False
counter = 0
step = 1

inten1 = start.Intensity(wifi)
while found is False and counter < interation:
    
    
    start.MoveAlongLine_xy(new_direction, step)
    inten2 = start.Intensity(wifi)
    
    if inten2 < inten1:
        step = -step/3
    
    if abs((inten2 - inten1) *100 / inten1) < 0.005:
        found = True
        
    inten1, inten2 = inten2, None
    
        
    

Intensity at x: 0, y: 0, z: 0, is I: 0.03005710850616171
Moved to x: 0.7071067811865476, y: -0.7071067811865475, z: 0
Intensity at x: 0.7071067811865476, y: -0.7071067811865475, z: 0, is I: 0.030409907117223297
Moved to x: 1.4142135623730951, y: -1.414213562373095, z: 0
Intensity at x: 1.4142135623730951, y: -1.414213562373095, z: 0, is I: 0.03075216057127642
Moved to x: 2.121320343559643, y: -2.1213203435596424, z: 0
Intensity at x: 2.121320343559643, y: -2.1213203435596424, z: 0, is I: 0.031082870699415663
Moved to x: 2.8284271247461903, y: -2.82842712474619, z: 0
Intensity at x: 2.8284271247461903, y: -2.82842712474619, z: 0, is I: 0.03140103819875777
Moved to x: 3.5355339059327378, y: -3.5355339059327373, z: 0
Intensity at x: 3.5355339059327378, y: -3.5355339059327373, z: 0, is I: 0.031705668906958986
Moved to x: 4.242640687119286, y: -4.242640687119285, z: 0
Intensity at x: 4.242640687119286, y: -4.242640687119285, z: 0, is I: 0.031995780432093605
Moved to x: 4.949747468305834, y:

In [15]:
new_direction_2 = new_direction.GetPerpendicularFromPoint(start, axis=2)

In [16]:
wifi_prediction = line.Line.IntersectLine(new_direction, new_direction_2, axis=2)

In [17]:
px = round(wifi_prediction.x, 0)
py1 = round(wifi_prediction.y, 0)
px, py1

(51.0, 23.0)

In [18]:
wifi.x, wifi.y, wifi.z

(51, 23, 14)

In [19]:
init_p1, init_p3 = point.Point(0, inity, initz), point.Point(0, 1, 1)

In [20]:
init_line_2 = line.Line(init_p1, init_p3, axis=3)

In [21]:
init_line_2.slope_yz, init_line_2.intercept_yz

(1.0, 0.0)

In [22]:
interation = 100
found = False
counter = 0
step = 1

inten1 = start.Intensity(wifi)
while found is False and counter < interation:
    
    
    start.MoveAlongLine_yz(init_line_2, step)
    inten2 = start.Intensity(wifi)
    
    if inten2 < inten1:
        step = -step/3
    
    if abs((inten2 - inten1) *100 / inten1) < 0.005:
        found = True
        
    inten1, inten2 = inten2, None
    
        
    

Intensity at x: 13.906433363335438, y: -13.906433363335436, z: 0, is I: 0.03407134699492366
Moved to x: 13.906433363335438, y: -13.199326582148888, z: 0.7071067811865475
Intensity at x: 13.906433363335438, y: -13.199326582148888, z: 0.7071067811865475, is I: 0.03491589705451454
Moved to x: 13.906433363335438, y: -12.49221980096234, z: 1.414213562373095
Intensity at x: 13.906433363335438, y: -12.49221980096234, z: 1.414213562373095, is I: 0.03577776092489325
Moved to x: 13.906433363335438, y: -11.785113019775793, z: 2.1213203435596424
Intensity at x: 13.906433363335438, y: -11.785113019775793, z: 2.1213203435596424, is I: 0.0366563566459319
Moved to x: 13.906433363335438, y: -11.078006238589245, z: 2.82842712474619
Intensity at x: 13.906433363335438, y: -11.078006238589245, z: 2.82842712474619, is I: 0.037550967427726185
Moved to x: 13.906433363335438, y: -10.370899457402697, z: 3.5355339059327373
Intensity at x: 13.906433363335438, y: -10.370899457402697, z: 3.5355339059327373, is I: 0

In [23]:
new_direction = init_line_2.GetPerpendicularFromPoint(start, axis=3)

In [24]:
new_direction.slope_yz, new_direction.intercept_yz

(-1.0, 36.69098520156862)

In [25]:
start.Goto(initx, inity, initz)

Moved to x: 0, y: 0, z: 0


In [26]:
interation = 100
found = False
counter = 0
step = 1

inten1 = start.Intensity(wifi)
while found is False and counter < interation:
    
    
    start.MoveAlongLine_yz(new_direction, step)
    inten2 = start.Intensity(wifi)
    
    if inten2 < inten1:
        step = -step/3
    
    if abs((inten2 - inten1) *100 / inten1) < 0.005:
        found = True
        
    inten1, inten2 = inten2, None
    
        
    

Intensity at x: 0, y: 0, z: 0, is I: 0.03005710850616171
Moved to x: 0, y: 0.7071067811865476, z: -0.7071067811865475
Intensity at x: 0, y: 0.7071067811865476, z: -0.7071067811865475, is I: 0.03016343686101855
Moved to x: 0, y: 1.4142135623730951, z: -1.414213562373095
Intensity at x: 0, y: 1.4142135623730951, z: -1.414213562373095, is I: 0.0302522051693665
Moved to x: 0, y: 2.121320343559643, z: -2.1213203435596424
Intensity at x: 0, y: 2.121320343559643, z: -2.1213203435596424, is I: 0.03032309653115186
Moved to x: 0, y: 2.8284271247461903, z: -2.82842712474619
Intensity at x: 0, y: 2.8284271247461903, z: -2.82842712474619, is I: 0.030375855848989504
Moved to x: 0, y: 3.5355339059327378, z: -3.5355339059327373
Intensity at x: 0, y: 3.5355339059327378, z: -3.5355339059327373, is I: 0.030410292105887325
Moved to x: 0, y: 4.242640687119286, z: -4.242640687119285
Intensity at x: 0, y: 4.242640687119286, z: -4.242640687119285, is I: 0.030426280086027963
Moved to x: 0, y: 4.949747468305834

In [27]:
new_direction_2 = new_direction.GetPerpendicularFromPoint(start, axis=3)

In [28]:
wifi_prediction = line.Line.IntersectLine(new_direction, new_direction_2, axis=3)

In [29]:
py2 = round(wifi_prediction.y, 0)
pz = round(wifi_prediction.z, 0)

In [30]:
px, py1, py2, pz

(51.0, 23.0, 23.0, 14.0)

In [31]:
wifi.x, wifi.y, wifi.z

(51, 23, 14)